In [14]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm

In [15]:
%store -r test

# Take the name of the different classes (e.g. '0', '1', '2', ...)
classes = test.groupby(['label'])['label'].unique().index.to_list()

In [16]:
predicted_test = test.copy()

feature_1= 'arctan_1'
feature_2='curvature_1'

models = []
for n_class in classes: # Can be thrown an expection beacuse this is the labels of the test set and not of the models (train set)
    model = pickle.load(open('models_8/'+str(n_class)+'hmm_model.pkl', 'rb'))
    models.append(model)

#log_likelihoods = pd.DataFrame()
#log_likelihoods['predicted_class'] = classes
#log_like_max = []
#top_3_loglikelihood = []
top_3_predicted_classes = []
predicted_class = []
#is_predicted = []
true_label=[]

for i in tqdm(range(len(test))):
    angle_1 = test[feature_1].iloc[i][1:-1]
    angle_2 = test[feature_2].iloc[i]
    X=np.array([angle_1,angle_2]).T
    # X = np.hstack([angle_1, angle_2]).T
    true_label.append(test['label'].iloc[i])

    list_log_likelihoods = []
    for model in models:
        
        list_log_likelihoods.append(model.score(X,lengths=len(X)))
        pass

    predicted_class.append(np.argmax(list_log_likelihoods))

    top_3_values = sorted(list_log_likelihoods)[-3:]
    top_3_predicted_classes.append([i for i, x in enumerate(list_log_likelihoods) if x == top_3_values[0] or x == top_3_values[1] or x == top_3_values[2]])
    
# model_0.score_samples(angles_0)

100%|██████████| 248/248 [00:01<00:00, 193.21it/s]


In [17]:
print(predicted_class)
print(true_label)

# print(len(predicted_class))
# print(len(true_label))

[9, 1, 9, 2, 6, 8, 0, 4, 3, 9, 8, 9, 4, 2, 0, 5, 3, 6, 4, 2, 9, 8, 9, 0, 3, 3, 0, 4, 6, 8, 4, 3, 2, 0, 0, 4, 2, 4, 1, 2, 2, 2, 3, 2, 7, 7, 5, 5, 8, 8, 7, 4, 5, 3, 3, 1, 0, 7, 5, 8, 0, 0, 6, 9, 9, 0, 4, 9, 3, 3, 5, 2, 8, 9, 3, 4, 4, 2, 6, 0, 3, 9, 0, 1, 2, 8, 1, 5, 0, 8, 4, 1, 8, 4, 4, 9, 0, 2, 2, 7, 3, 9, 6, 8, 5, 6, 6, 1, 1, 1, 4, 0, 9, 3, 4, 3, 4, 9, 3, 4, 9, 6, 9, 6, 0, 7, 1, 3, 0, 6, 9, 2, 7, 4, 8, 8, 8, 4, 8, 9, 8, 3, 9, 4, 7, 8, 7, 3, 9, 4, 4, 9, 7, 6, 7, 8, 9, 0, 0, 7, 9, 6, 9, 4, 4, 0, 8, 8, 0, 4, 4, 5, 9, 6, 8, 3, 9, 7, 9, 0, 4, 1, 4, 2, 8, 9, 2, 0, 3, 0, 6, 9, 2, 6, 8, 1, 6, 8, 3, 9, 2, 6, 0, 1, 7, 2, 1, 6, 3, 2, 6, 8, 0, 4, 2, 7, 0, 7, 0, 3, 7, 0, 2, 2, 5, 6, 5, 9, 6, 3, 1, 4, 4, 2, 6, 3, 9, 4, 0, 3, 9, 6, 7, 2, 4, 9, 6, 6]
[4, 1, 9, 3, 5, 8, 0, 4, 6, 0, 8, 4, 7, 2, 6, 4, 7, 3, 1, 2, 8, 0, 9, 0, 5, 3, 6, 4, 5, 2, 4, 5, 2, 0, 0, 4, 2, 1, 2, 9, 3, 2, 9, 3, 1, 7, 5, 5, 8, 6, 7, 4, 5, 3, 3, 1, 0, 9, 6, 8, 9, 0, 6, 9, 9, 8, 1, 0, 2, 4, 5, 2, 6, 9, 3, 4, 4, 2, 6, 0, 9, 9, 8, 7, 2,

In [18]:
from sklearn.metrics.cluster import contingency_matrix 


In [19]:
matrix=contingency_matrix(true_label, predicted_class)

In [20]:
matrix

array([[20,  0,  0,  1,  0,  0,  1,  0,  6,  6],
       [ 0, 11,  0,  0,  9,  0,  1,  2,  0,  0],
       [ 0,  1, 16,  2,  1,  0,  1,  0,  1,  1],
       [ 0,  0,  7, 10,  1,  0,  2,  0,  0,  2],
       [ 0,  0,  0,  1, 19,  2,  0,  0,  0,  3],
       [ 1,  0,  0,  5,  1,  7,  8,  0,  0,  3],
       [ 4,  1,  1,  2,  0,  1, 10,  0,  5,  1],
       [ 1,  2,  0,  2,  2,  0,  0, 13,  0,  1],
       [ 3,  0,  0,  1,  0,  0,  2,  0, 14,  4],
       [ 1,  0,  2,  3,  1,  1,  1,  3,  0, 14]])

In [21]:
accuracy=np.sum(np.diag(matrix))/np.sum(matrix)
print("Accuracy: ", accuracy)

Accuracy:  0.5403225806451613


In [22]:
# Calculate top 3 accuracy
sum_top_3 = 0
for i in range(len(top_3_predicted_classes)):
    if true_label[i] in top_3_predicted_classes[i]:
        sum_top_3 += 1

accuracy_top_3 = sum_top_3/len(predicted_test["label"])
print("Accuracy top 3: ", accuracy_top_3)

Accuracy top 3:  0.8588709677419355
